# Ensemble модель

- Комбинация GARCH и LightGBM
- Взвешенное прогнозирование (среднее)
- Сравнение по метрикам, бенчмаркинг

Готовая заготовка для интеграции обоих блоков и финального скрипта.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")


## Загрузка прогнозов всех моделей


In [ ]:
MODEL_DIR = Path('data') / 'models'
ticker = 'SBER'

# Загрузка результатов baseline моделей
try:
    baseline_df = pd.read_parquet(MODEL_DIR / f"{ticker}_baseline_forecasts.parquet")
    print(f"✅ Baseline модели загружены: {len(baseline_df)} записей")
except FileNotFoundError:
    print("⚠️ Baseline модели не найдены. Запустите 01_baseline_models.ipynb")
    baseline_df = None

# Загрузка результатов GARCH
try:
    garch_df = pd.read_parquet(MODEL_DIR / f"{ticker}_garch_forecasts.parquet")
    print(f"✅ GARCH модель загружена: {len(garch_df)} прогнозов")
except FileNotFoundError:
    print("⚠️ GARCH модель не найдена. Запустите 02_garch_model.ipynb")
    garch_df = None

# Загрузка результатов LightGBM
try:
    lgb_df = pd.read_parquet(MODEL_DIR / f"{ticker}_lgb_quantile_forecasts.parquet")
    print(f"✅ LightGBM модель загружена: {len(lgb_df)} прогнозов")
except FileNotFoundError:
    print("⚠️ LightGBM модель не найдена. Запустите 03_lightgbm_quantile.ipynb")
    lgb_df = None


## Создание ансамбля


In [ ]:
def create_ensemble(baseline, garch, lgb, weights=None):
    """
    Создание ансамбля моделей с взвешиванием
    weights: [baseline_weight, garch_weight, lgb_weight]
    По умолчанию - равные веса
    """
    if weights is None:
        weights = [1/3, 1/3, 1/3]
    
    ensemble = []
    
    # Если доступны все модели
    if baseline is not None and garch is not None and lgb is not None:
        ensemble = (baseline * weights[0] + 
                   garch * weights[1] + 
                   lgb * weights[2])
        return ensemble
    
    # Если доступны только baseline и GARCH
    elif baseline is not None and garch is not None:
        w = weights[0] / (weights[0] + weights[1])
        ensemble = baseline * w + garch * (1 - w)
        return ensemble
    
    # Если доступны только baseline и LGB
    elif baseline is not None and lgb is not None:
        w = weights[0] / (weights[0] + weights[2])
        ensemble = baseline * w + lgb * (1 - w)
        return ensemble
    
    # Если доступен только baseline
    elif baseline is not None:
        return baseline
    
    else:
        return None


# Пример: создаем ансамбль на основе доступных моделей
if baseline_df is not None:
    # Используем EWMA baseline как пример
    baseline_pred = baseline_df['forecast_ewma'].values
    garch_pred = garch_df['garch_forecast'].values if garch_df is not None else None
    lgb_pred = lgb_df['pred_q50'].values if lgb_df is not None else None
    
    # Выравниваем размерности (используем минимальную длину)
    if garch_pred is not None and lgb_pred is not None:
        min_len = min(len(baseline_pred), len(garch_pred), len(lgb_pred))
        baseline_pred = baseline_pred[-min_len:]
        garch_pred = garch_pred[-min_len:]
        lgb_pred = lgb_pred[-min_len:]
    
    # Создаем ансамбль с весами [0.2, 0.3, 0.5] (больше веса LightGBM)
    ensemble_forecast = create_ensemble(baseline_pred, garch_pred, lgb_pred, weights=[0.2, 0.3, 0.5])
    
    print(f"\n✅ Ансамбль создан: {len(ensemble_forecast)} прогнозов")
    print(f"Среднее значение прогноза: {ensemble_forecast.mean():.4f}")
    
else:
    print("❌ Недостаточно данных для создания ансамбля")


## Сравнение метрик всех моделей


In [ ]:
def calculate_metrics(actual, predicted):
    """Расчет метрик качества"""
    mask = ~(np.isnan(actual) | np.isnan(predicted))
    actual_clean = actual[mask]
    predicted_clean = predicted[mask]
    
    mae = np.mean(np.abs(actual_clean - predicted_clean))
    rmse = np.sqrt(np.mean((actual_clean - predicted_clean)**2))
    mape = np.mean(np.abs((actual_clean - predicted_clean) / actual_clean)) * 100
    
    return {'mae': mae, 'rmse': rmse, 'mape': mape}


# Получаем актуальные значения для сравнения
if baseline_df is not None and ensemble_forecast is not None:
    actual_values = baseline_df['target_vol_5d'].values[-len(ensemble_forecast):]
    
    # Метрики для каждой модели
    metrics_comparison = []
    
    # Baseline (EWMA)
    baseline_metrics = calculate_metrics(actual_values, baseline_pred)
    metrics_comparison.append({'model': 'Baseline (EWMA)', **baseline_metrics})
    
    # GARCH
    if garch_pred is not None:
        garch_metrics = calculate_metrics(actual_values, garch_pred)
        metrics_comparison.append({'model': 'GARCH(1,1)', **garch_metrics})
    
    # LightGBM
    if lgb_pred is not None:
        lgb_metrics = calculate_metrics(actual_values, lgb_pred)
        metrics_comparison.append({'model': 'LightGBM', **lgb_metrics})
    
    # Ensemble
    ensemble_metrics = calculate_metrics(actual_values, ensemble_forecast)
    metrics_comparison.append({'model': 'Ensemble', **ensemble_metrics})
    
    # Создаем таблицу
    metrics_df = pd.DataFrame(metrics_comparison)
    
    print(f"\n=== Сравнение моделей ({ticker}) ===")
    print(metrics_df.to_string(index=False))
    
    # Лучшая модель
    best_model = metrics_df.loc[metrics_df['mae'].idxmin(), 'model']
    print(f"\n🏆 Лучшая модель по MAE: {best_model}")
    
else:
    print("❌ Недостаточно данных для сравнения метрик")


## Сохранение результатов


In [ ]:
if ensemble_forecast is not None:
    OUTPUT_DIR = Path('data') / 'models'
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    # Сохранение ансамбля
    ensemble_df = pd.DataFrame({
        'actual': actual_values,
        'ensemble_forecast': ensemble_forecast,
        'baseline_forecast': baseline_pred,
        'garch_forecast': garch_pred if garch_pred is not None else np.nan,
        'lgb_forecast': lgb_pred if lgb_pred is not None else np.nan
    })
    
    output_path = OUTPUT_DIR / f"{ticker}_ensemble_forecasts.parquet"
    ensemble_df.to_parquet(output_path, index=False)
    
    # Сохранение метрик
    metrics_path = OUTPUT_DIR / f"{ticker}_models_comparison.csv"
    metrics_df.to_csv(metrics_path, index=False)
    
    print(f"\n✅ Ансамбль сохранен: {output_path}")
    print(f"✅ Сравнение метрик: {metrics_path}")
    
else:
    print("⚠️ Нечего сохранять")
